In [1]:
import pandas as pd
import numpy as np
import ast
import re
from zhon.hanzi import non_stops, stops
import os
import pickle
from opencc import OpenCC
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import RobertaTokenizer
from transformers import BertTokenizer , BertConfig , BertModel , XLNetTokenizer, XLNetConfig , XLNetModel

import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification,BertForPreTraining
from torch.autograd import Variable

from torch import nn


In [2]:
# train_df = pd.read_csv("./tbrain/tbrain_train_split.csv")
train_df = pd.read_csv("./tbrain/tbrain_train.csv")

train_df = train_df.fillna('None')

# test_df = pd.read_csv("./tbrain/tbrain_test_split.csv")
test_df = pd.read_csv("./tbrain/tbrain_test.csv")

train_df = pd.concat([train_df,test_df])

test_df = test_df.fillna('None')

print(train_df.shape)
print(test_df.shape)


(4917, 4)
(491, 4)


In [3]:
def clean_string(content):
#     cc = OpenCC('t2s')
    content = content.replace('\n','。').replace('\t','，').replace('!', '！').replace('?', '？')# erease white space cause English name error
    content = re.sub("[+\.\/_,$%●▼►^*(+\"\']+|[+——~@#￥%……&*（）★]", "",content)
    content = re.sub(r"[%s]+" %stops, "。",content)
#     content = cc.convert(content)
    return content

def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para) 
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    return para.split("\n")


def combine_sentence(sentences):
    li = []
    string = ''
    for k in range(len(sentences)):
        sentence = sentences[k]
        if len(string) + len(sentence) < 510:
            string = string + sentence
        else:
#             原本是空的代表sentences太常
            if string == '':
                n = 510
                tmp_li = [sentence[i:i+n] for i in range(0, len(sentence), n)]
                string = tmp_li.pop(-1)
                li = li + tmp_li
            else:
                li.append(string)
                string = sentence
    if(string != ''):
        li.append(string)
    return li
            
    
    

In [4]:
names =  train_df['name'].tolist()
contents = np.array(train_df['full_content'].tolist())
ckip_names = train_df['ckip_names'].tolist()
x = []
binary_y = []
names_y  = []
content_all_names = []
start_pos_labels = []
end_pos_labels = []
for i in range(len(contents)):
    content = contents[i]
    content = clean_string(content)
    content_ckip_names = ast.literal_eval(ckip_names[i])

    if(content=='nan'):
        continue

    
    name_li  = ast.literal_eval(names[i])
    if(len(name_li) == 0 ):
#         content切句 
#         split_content = cut_sent(content)
#         chunks = combine_sentence(split_content)
        
#         for chunk in chunks:
#             start_pos_label = np.zeros(512)
#             end_pos_label = np.zeros(512)
#             binary_y.append(0)
#             start_pos_label[0] = 1
#             end_pos_label[0] = 1
#             x.append(chunk)
#             start_pos_labels.append(start_pos_label)
#             end_pos_labels.append(end_pos_label)
#             content_all_names.append(content_ckip_names)
        binary_y.append(0)
        x.append(content)
#         if len(content)  <= 512:
#             x.append(content)
#         else:
#             x.append(content[-512:])
        
    else:
        binary_y.append(1)
        x.append(content)

#         if len(content)  <= 512:
#             x.append(content)
#         else:
#             x.append(content[-512:])
        
#         for name in name_li:
        
#             name = name.replace('\n','').replace('\t','').replace(' ','')
#             #  content切句 
            
#             _pos = 0
#             while True:
#                 start_pos = content.find(name,_pos)
#                 if(start_pos == -1):
#                     break
#                 start_pos += 1
#                 _pos = start_pos
#                 end_pos = start_pos + len(name)

#                 if(start_pos < 512 and end_pos < 512 ):
#                     binary_y.append(1)
#                     x.append(content)

#                     start_pos_label = np.zeros(512)
#                     end_pos_label = np.zeros(512)
#                     start_pos_label[start_pos] = 1
#                     end_pos_label[end_pos] = 1

#                     start_pos_labels.append(start_pos_label)
#                     end_pos_labels.append(end_pos_label)
#                     content_all_names.append(content_ckip_names)
        

        
        
train_x = np.array(x)
train_binary_y = np.array(binary_y)
train_start_pos_labels = np.array(start_pos_labels)
train_end_pos_labels = np.array(end_pos_labels)
train_ckip_names = np.array(content_all_names)


print(train_x.shape)
print(train_binary_y.shape)
print(train_start_pos_labels.shape)
print(train_end_pos_labels.shape)
print(train_ckip_names.shape)

(4917,)
(4917,)
(0,)
(0,)
(0,)


In [5]:
model_path  = './chinese_xlnet_mid_pytorch/'

tokenizer = XLNetTokenizer.from_pretrained(model_path)

train_input_dict = tokenizer.batch_encode_plus(train_x, 
                                         add_special_tokens=True,
                                         max_length=512,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [6]:
class TrainDataset(Dataset):
    def __init__(self, input_dict, y):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y

    def __getitem__(self,idx):
        inputid = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return inputid , tokentype , attentionmask, y
    
    def __len__(self):
        return len(self.input_ids)
    

In [7]:
from transformers import XLNetForSequenceClassification
BATCH_SIZE = 3
trainset = TrainDataset(train_input_dict, 
                        train_binary_y)

trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

NUM_LABELS = 2
model_path  = './chinese_xlnet_mid_pytorch/'
model = XLNetForSequenceClassification.from_pretrained(model_path,num_labels=NUM_LABELS)
model = model.to(device)
model.train()



EPOCHS = 5
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


for epoch in range(0,EPOCHS):
    running_loss = 0.0
    correct = 0
    total = 0

    for data in trainloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors , binary_y  = [t.to(device) for t in data]
        
        optimizer.zero_grad()
        outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors,
                                labels =binary_y )
        
  

        loss = outputs[0]
        logits = outputs[1]
        loss.backward()
        optimizer.step()
        
        total += len(binary_y)
        logits = torch.argmax(logits, dim=-1)
        correct += (logits == binary_y).sum().item()
        running_loss += loss.item()
        
    torch.save(model.state_dict(),'./TB_multispan/XLNet_fromseq_' + str(epoch) + '.pkl')
    print(epoch)
    print('train acc:' , correct/total)
    print(running_loss)


device: cuda:0


Some weights of the model checkpoint at ./chinese_xlnet_mid_pytorch/ were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at ./chinese_xlnet_mid_pytorch/ and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to 

0
train acc: 0.9778320113890584
105.24525765707222
1
train acc: 0.9945088468578401
24.691656853685345
2
train acc: 0.9971527354077689
21.938064086025406
3
train acc: 0.998983119788489
8.773084191643648


KeyboardInterrupt: 

In [ ]:
def xlnet_news_has_ans(news):
    class Testset(Dataset):
        def __init__(self, input_dict):
            self.input_ids = input_dict['input_ids']
            self.token_type_ids = input_dict['token_type_ids']
            self.attention_mask = input_dict['attention_mask']
        def __getitem__(self,idx):
            inputid = self.input_ids[idx]
            tokentype = self.token_type_ids[idx]
            attentionmask = self.attention_mask[idx]
            return inputid , tokentype , attentionmask

        def __len__(self):
            return len(self.input_ids)
    
    lm_path = './chinese_xlnet_mid_pytorch/'
    tokenizer = BertTokenizer.from_pretrained(lm_path)

    content = clean_string(news)
    tokenizer = XLNetTokenizer.from_pretrained(model_path)

    input_dict = tokenizer.batch_encode_plus([content], 
                                         add_special_tokens=True,
                                         max_length=512,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')
    
        

    
    testset = Testset(input_dict)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    
    from transformers import XLNetForSequenceClassification
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    NUM_LABELS = 2
    model = XLNetForSequenceClassification.from_pretrained(lm_path,num_labels=NUM_LABELS)
    check_point = './TB_multispan/XLNet_fromseq_0.pkl'
    model.load_state_dict(torch.load(check_point))
    model = model.to(device)
    model.eval()


    with torch.no_grad():
        for data in testloader:
            tokens_tensors, segments_tensors, masks_tensors = [t.to(device) for t in data]
            outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
            pred = torch.softmax(outputs[0] , dim = -1)
            torch.set_printoptions(precision=10)
            pred = torch.argmax(pred,dim=-1)
            pred = pred.cpu().detach().numpy()
            return pred
        
    
    
    
    

In [12]:
EPOCHS = 20
for epoch in range(10,EPOCHS):
    running_loss = 0.0
    correct = 0
    total = 0

    for data in trainloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors , binary_y  = [t.to(device) for t in data]
        
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        
  

        bert_all_768 = bert_outputs[0]
        optimizer.zero_grad()
        
        total += bert_all_768.size()[0]
        
        
        logits = binary_model(bert_all_768.double())
        loss = loss_fn(logits , binary_y)
        loss.backward()
        optimizer.step()
        
        
        logits = torch.argmax(logits, dim=-1)
        correct += (logits == binary_y).sum().item()
        running_loss += loss.item()
        
    torch.save(binary_model.state_dict(),'./TB_multispan/XLNet_binary_alldataset_4k_' + str(epoch) + '.pkl')
    print(epoch)
    print('train acc:' , correct/total)

NameError: name 'optimizer' is not defined

In [12]:
names =  test_df['name'].tolist()
contents = np.array(test_df['full_content'].tolist())
ckip_names = test_df['ckip_names'].tolist()

x = []
binary_y = []
names_y  = []
content_all_names = []

start_pos_labels = []
end_pos_labels = []
for i in range(len(contents)):
    content = contents[i]
    content = clean_string(content)
    content_ckip_names = ast.literal_eval(ckip_names[i])


    if(content=='nan'):
        continue
#     name = names[i]
    name_li  = ast.literal_eval(names[i])


    if(len(name_li) == 0):
        
        start_pos_label = np.zeros(512)
        end_pos_label = np.zeros(512)
        binary_y.append(0)
        start_pos_label[0] = 1
        end_pos_label[0] = 1
        x.append(content)
        start_pos_labels.append(start_pos_label)
        end_pos_labels.append(end_pos_label)
        content_all_names.append(content_ckip_names)
        
    else:
        
        for name in name_li:
        
            name = name.replace('\n','').replace('\t','').replace(' ','')

            start_pos = chunk.find(name)
            if(start_pos == -1):
                continue
            start_pos += 1
            _pos = start_pos
            end_pos = start_pos + len(name)

            if(start_pos < 512 and end_pos < 512 ):
                binary_y.append(1)
                x.append(chunk)

                start_pos_label = np.zeros(512)
                end_pos_label = np.zeros(512)
                start_pos_label[start_pos] = 1
                end_pos_label[end_pos] = 1

                start_pos_labels.append(start_pos_label)
                end_pos_labels.append(end_pos_label)
                content_all_names.append(content_ckip_names)
        

    
test_x = np.array(x)
test_binary_y = np.array(binary_y)
test_start_pos_labels = np.array(start_pos_labels)
test_end_pos_labels = np.array(end_pos_labels)
test_ckip_names = np.array(content_all_names)


print(test_x.shape)
print(test_binary_y.shape)
print(test_start_pos_labels.shape)
print(test_end_pos_labels.shape)
print(test_ckip_names.shape)

(454,)
(454,)
(454, 512)
(454, 512)
(454,)


In [13]:
model_path  = './chinese_xlnet_mid_pytorch/'

tokenizer = XLNetTokenizer.from_pretrained(model_path)

test_input_dict = tokenizer.batch_encode_plus(test_x, 
                                         add_special_tokens=True,
                                         max_length=512,
                                         return_special_tokens_mask=True,
                                         pad_to_max_length=True,
                                         return_tensors='pt')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [14]:
BATCH_SIZE = 20
testset = TrainDataset(test_input_dict, 
                        test_binary_y,
                        test_start_pos_labels,
                        test_end_pos_labels)

testloader = DataLoader(testset, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print("device:", device)

model_path  = './chinese_xlnet_mid_pytorch/'
config = XLNetConfig.from_pretrained(model_path + 'config.json',output_hidden_states=True)
model = XLNetModel.from_pretrained(model_path,config=config)
model = model.to(device)
model.eval()

binary_model_path = './TB_multispan/XLNet_binary16.pkl'
checkpoint = torch.load(binary_model_path)
binary_model = XLNetBinrayClassifier(config)
binary_model.load_state_dict(checkpoint)
binary_model = binary_model.to(device)
binary_model.eval()





with torch.no_grad():
    running_loss = 0.0
    correct = 0
    total = 0
    
    for data in testloader:
        
        tokens_tensors ,  segments_tensors , masks_tensors , binary_y ,start_pos_labels , end_pos_labels = [t.to(device) for t in data]
        
        bert_outputs = model(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)
        
  

        bert_all_768 = bert_outputs[0]
    
        total += bert_all_768.size()[0]
        logits = binary_model(bert_all_768)

        
        
        logits = torch.argmax(logits, dim=-1)
        correct += (logits == binary_y).sum().item()
        
    print('train acc:' , correct/total)


            



device: cuda:0
train acc: 0.9845814977973568
